In [30]:
import mca

In [31]:
import pandas as pd
import numpy as np

In [32]:
%matplotlib inline
import matplotlib.pyplot as plt

In [33]:
import os

In [34]:
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold

In [38]:
os.chdir(r'D:\Matias\Python\DH\bases_enprecosp2011')
with open(r'Base Usuario ENPreCoSP-2011.txt', 'rb') as data0:
    data0 = data0.read().decode()
data_spl = data0.split()
df0 = pd.DataFrame([i.split('|')[:-1] for i in data_spl[1:]], columns = [i for i in data_spl[0].split('|')])

In [39]:
#alguna info
#deberia decir nulos?
print (df0.shape)
df0.info()
df0.head()

(34343, 291)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34343 entries, 0 to 34342
Columns: 291 entries, IDENTIFI to BIDT03
dtypes: object(291)
memory usage: 76.2+ MB


,IDENTIFI,W_PERS,REGION,POB_URB,PRVNC,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,...,BIDT02_05,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03
0,61301445099,3787,1,4,6,1,3,4,1,1,...,,,,,,,,,,
1,94924403348,5301,1,4,6,1,3,4,1,1,...,,,,,,,,,,
2,63798413037,2248,1,4,6,1,3,4,1,1,...,,,,,,,,,,
3,75477993771,6157,1,4,6,1,3,3,1,1,...,,,,,,,,,,
4,34764150916,12528,1,4,6,1,3,4,1,1,...,,,,,,,,,,


In [40]:
#identificando indices
df0.iloc[:1,6:17]

,BHCV01,BHCV02,BHCV03,BHCV04,BHCV05,BHCV06,BHCV07,BHCV08,BHCV09,BHCV10,BHCV11
0,3,4,1,1,1,1,1,1,1,1,1


In [41]:
#para agarrar todas las variables que no sean de consumo, los índices usados serán [2:66]

# Target

Vamos a construir una variable target que significaría consumo problemático
Agrupamos criterios para las distintas drogas y si cumple con alguno de ellos es un caso positivo.

In [42]:
df0 = df0.replace(['9', '99', '999', '888'], np.nan) #estos son las respuesas de "no sabe/no contesta"
df0 = df0.apply(pd.to_numeric, errors = 'coerce')
df0 = df0.fillna(0)

In [43]:
#OJO CON RESPUESTAS 9, 99, 999, 888
def tabaco(row):
    if (row['BITA05'] > 10 ) | (row['BITA06'] > 0 ):
        return 1
    else:
        return 0
def marihuana(row):
    if ((row['BIMA04'] == 1) | (row['BIMA06'] == 1) | (row['BIMA11'] == 1) | (row['BIMA12'] == 1) |
        (row['BIMA13'] == 1) | (row['BIMA16'] == 1)):
        return 1
    else:
        return 0
def alcohol(row):
    if ((row['AB_TC'] == 1) | (row[['BIBA07_01', 'BIBA07_02', 'BIBA07_03']].sum() > 20) |
    (row[['BIBA08_01', 'BIBA08_02', 'BIBA08_03', 'BIBA09_01', 'BIBA09_02', 'BIBA09_03']].sum() > 20) |
    (row['BIBA12'] == 1) | (row['BIBA06'] >= 3) | (row['BIBA11'] > 9)): 
        return 1
    else:
        return 0
def cocaina(row):
    if ((row['BICO04'] == 1) | (row['BICO06'] == 1) | (row['BICO11'] == 1) | 
        (row['BICO12'] == 1) | (row['BICO13'] == 1) | (row['BICO14'] == 1) | (row['BICO16'] == 1)):
        return 1
    else:
        return 0
def pastabase(row):
    if ((row['BIPB04'] == 1) | (row['BIPB06'] == 1) | (row['BIPB11'] == 1) | 
        (row['BIPB12'] == 1) | (row['BIPB13'] == 1) | (row['BIPB14'] == 1) | (row['BIPB16'] == 1)):
        return 1
    else:
        return 0
def extasis(row):
    if ((row['BIEX04'] == 1) | (row['BIEX06'] == 1) | (row['BIEX11'] == 1) | 
        (row['BIEX12'] == 1) | (row['BIEX13'] == 1) | (row['BIEX15'] == 1) | 
        (row['BIEX14'] == 1) | (row['BIEX16'] == 1)):
        return 1
    else:
        return 0
def otras(row):
    if ((row['BIES08'] >= 3) | (row['BITR08'] >= 10) | (row['BIIN01'] == 1)):
        return 1
    else:
        return 0
def tratamientos(row):
    if ((row['BIDT01'] == 1) | (row['BIDT03'] in {1, 2})):
        return 1
    else:
        return 0

In [44]:
df0['Tabaco'] = df0.apply(tabaco, axis = 1)
df0['Marihuana'] = df0.apply(marihuana, axis = 1)
df0['Cocaina'] = df0.apply(cocaina, axis = 1)
df0['Extasis'] = df0.apply(extasis, axis = 1)
df0['Alcohol'] = df0.apply(alcohol, axis = 1)
df0['Otras'] = df0.apply(otras, axis = 1)
df0['Tratamiento'] = df0.apply(tratamientos, axis = 1)

In [45]:
df_target = df0['Tabaco, Marihuana, Cocaina, Extasis, Alcohol, Otras'.split(', ')]
df_target['Total'] = df_target.sum(axis = 1)
df_target['Binario'] = [1 if t > 0 else 0 for t in df_target['Total']]

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [46]:
df_target.head()

,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras,Total,Binario
0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,1
2,1,0,0,0,0,0,1,1
3,1,0,0,0,1,0,2,1
4,1,0,0,0,1,0,2,1


Creo una tabla de target para agregar a los dummies

In [47]:
df_target_d = df_target.drop(['Total', 'Binario'], axis=1)
df_target_d.head()

,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,1,0
4,1,0,0,0,1,0


In [48]:
df_target_d.shape

(34343, 6)

### Get Dummies

In [136]:
dfd = pd.get_dummies(df0.iloc[:,2:66])

In [49]:
dfd.head()

,REGION,POB_URB,PRVNC,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,BHCV05,BHCV06,...,BIDT02_12,BIDT02_13,BIDT03,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras,Tratamiento
0,1,4,6,1,3,4.0,1,1.0,1.0,1,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,1,4,6,1,3,4.0,1,1.0,1.0,1,...,0.0,0.0,0.0,1,0,0,0,0,0,0
2,1,4,6,1,3,4.0,1,1.0,1.0,1,...,0.0,0.0,0.0,1,0,0,0,0,0,0
3,1,4,6,1,3,3.0,1,1.0,1.0,1,...,0.0,0.0,0.0,1,0,0,0,1,0,0
4,1,4,6,1,3,4.0,1,1.0,1.0,1,...,0.0,0.0,0.0,1,0,0,0,1,0,0


Segmento las variables en conjuntos para después analizarlas cruzándolas con las drogas

In [53]:
#Características Geográficas
dfd_geo0 = dfd.iloc[:,:42] #selecciono caracteristicas de df de dummies
dfd_geo = pd.concat([dfd_geo0, df_target_d], axis=1) #concateno con df de target para dummies
dfd_geo.head()

,REGION,POB_URB,PRVNC,AGL_URB,BHCV01,BHCV02,BHCV03,BHCV04,BHCV05,BHCV06,...,BHCH04,BHCH05,GRUPEDAD,BHCH07,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,1,4,6,1,3,4.0,1,1.0,1.0,1,...,2,16,2,6,0,0,0,0,0,0
1,1,4,6,1,3,4.0,1,1.0,1.0,1,...,1,59,5,2,1,0,0,0,0,0
2,1,4,6,1,3,4.0,1,1.0,1.0,1,...,2,65,5,4,1,0,0,0,0,0
3,1,4,6,1,3,3.0,1,1.0,1.0,1,...,1,39,4,2,1,0,0,0,1,0
4,1,4,6,1,3,4.0,1,1.0,1.0,1,...,1,19,2,6,1,0,0,0,1,0


In [54]:
#Características de Vivienda
dfd_viv0 = dfd.iloc[:,42:102]
dfd_viv = pd.concat([dfd_viv0, df_target_d], axis=1) #concateno con df de target para dummies
dfd_viv.head()

,NIVINSTR,COB_SALUD,CONDACT,CAT_OCUP,BISL12,BISL13,BISG01,BISG02,BISG03,BISG04,...,P1M_BA,ABU_C,ABU_V,ABU_W,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,4,3.0,3,0.0,0.0,0.0,2,2.0,2.0,2.0,...,2,0.0,0.0,0.0,0,0,0,0,0,0
1,3,3.0,1,2.0,2.0,0.0,4,2.0,2.0,2.0,...,2,0.0,0.0,0.0,1,0,0,0,0,0
2,5,3.0,1,3.0,1.0,1.0,3,2.0,2.0,1.0,...,2,0.0,0.0,0.0,1,0,0,0,0,0
3,5,3.0,2,0.0,0.0,0.0,3,2.0,2.0,2.0,...,1,2.0,2.0,2.0,1,0,0,0,1,0
4,6,3.0,1,3.0,2.0,0.0,2,2.0,2.0,2.0,...,1,2.0,2.0,2.0,1,0,0,0,1,0


In [55]:
#Características del Hogar
dfd_hogar0 = dfd.iloc[:,102:203]
dfd_hogar = pd.concat([dfd_hogar0, df_target_d], axis=1) #concateno con df de target para dummies
dfd_hogar.head()

,AB_ES,AB_FS,AB_TC,CE_EXC_C,CE_EXC_V,CE_EXC_W,CE_EXC_ES,CE_EXC_FS,CE_EXC_TC,BITR01,...,BIPB11,BIPB12,BIPB13,BIPB14,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1,0,0,0,1,0
4,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1,0,0,0,1,0


In [56]:
#Características del Jefe del Hogar
dfd_jefe0 = dfd.iloc[:,203:337]

In [57]:
#identificando indices de las edades no discretizadas
dfd_jefe0.iloc[:1,2:86]

,BIPB17_01,BIPB17_02,PV_PB,P1A_PB,P1M_PB,BIEX01,BIEX02,BIEX03,BIEX04,BIEX05,...,BIDT02_05,BIDT02_06,BIDT02_07,BIDT02_08,BIDT02_09,BIDT02_10,BIDT02_11,BIDT02_12,BIDT02_13,BIDT03
0,3.0,3.0,2,2,2,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [213]:
dfd_jefe1 = dfd_jefe0.drop(dfd_jefe0.iloc[:,2:86], axis=1) #dropeo las edades no discretizadas y me quedo con grupedad.
dfd_jefe = pd.concat([dfd_jefe1, df_target_d], axis=1) #concateno con df de target para dummies
dfd_jefe.head()

,BHCH04J_1,BHCH04J_2,GRUPEDAD_J_1,GRUPEDAD_J_2,GRUPEDAD_J_3,GRUPEDAD_J_4,GRUPEDAD_J_5,GRUPEDAD_J_6,GRUPEDAD_J_9,BHCH07J_1,...,BHSL12_9,BHSL13_,BHSL13_1,BHSL13_2,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0


In [303]:
#Características de persona seleccionado
dfd_pers0 = dfd.iloc[:,337:443]

In [205]:
#identificando indices de las edades no discretizadas
dfd_pers0.iloc[:1,12:62]

,BHCH05_16,BHCH05_17,BHCH05_18,BHCH05_19,BHCH05_20,BHCH05_21,BHCH05_22,BHCH05_23,BHCH05_24,BHCH05_25,...,BHCH05_56,BHCH05_57,BHCH05_58,BHCH05_59,BHCH05_60,BHCH05_61,BHCH05_62,BHCH05_63,BHCH05_64,BHCH05_65
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
dfd_pers1 = dfd_pers0.drop(dfd_pers0.iloc[:,12:62], axis=1) #dropeo las edades no discretizadas y me quedo con grupedad.
dfd_pers = pd.concat([dfd_pers1, df_target_d], axis=1) #concateno con df de target para dummies
dfd_pers.head()

,BHCH03_1,BHCH03_10,BHCH03_2,BHCH03_3,BHCH03_4,BHCH03_5,BHCH03_6,BHCH03_7,BHCH03_8,BHCH03_9,...,BISL12_9,BISL13_,BISL13_1,BISL13_2,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0


In [216]:
#Autopercepción de salud y entorno de la persona seleccionada
dfd_autop0 = dfd.iloc[:,443:503]
dfd_autop = pd.concat([dfd_autop0, df_target_d], axis=1) #concateno con df de target para dummies
dfd_autop.head()

,BISG01_1,BISG01_2,BISG01_3,BISG01_4,BISG01_5,BISG02_1,BISG02_2,BISG02_9,BISG03_1,BISG03_2,...,BIAC07_06_1,BIAC07_06_2,BIAC07_06_3,BIAC07_06_9,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
0,0,1,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,1,0


### Trial con un Sample

Pruebo con un sample de las variables geográficas (por cuestión de memoria)

In [296]:
rows = np.random.choice(dfd_geo.index.values, 1000)
geo_sample = dfd1.ix[rows]

In [297]:
geo_sample #hago un sample porque no me da la memoria para correr todo.

,REGION_1,REGION_2,REGION_3,REGION_4,REGION_5,REGION_6,POB_URB_1,POB_URB_2,POB_URB_3,POB_URB_4,...,BIAC07_06_1,BIAC07_06_2,BIAC07_06_3,BIAC07_06_9,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
19312,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
7242,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
8042,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
6392,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
21789,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
6118,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
18763,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
30456,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
16476,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
30250,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [298]:
geo_sample.shape

(1000, 509)

In [299]:
geo_sample.replace([np.inf, -np.inf], np.nan)
geo_sample.replace(r'\s+', np.nan, regex=True)

,REGION_1,REGION_2,REGION_3,REGION_4,REGION_5,REGION_6,POB_URB_1,POB_URB_2,POB_URB_3,POB_URB_4,...,BIAC07_06_1,BIAC07_06_2,BIAC07_06_3,BIAC07_06_9,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
19312,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
7242,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
8042,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
6392,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
21789,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
6118,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
18763,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
30456,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
16476,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
30250,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [300]:
geo_sample.fillna(0)

,REGION_1,REGION_2,REGION_3,REGION_4,REGION_5,REGION_6,POB_URB_1,POB_URB_2,POB_URB_3,POB_URB_4,...,BIAC07_06_1,BIAC07_06_2,BIAC07_06_3,BIAC07_06_9,Tabaco,Marihuana,Cocaina,Extasis,Alcohol,Otras
19312,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
7242,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
8042,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
6392,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
21789,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
6118,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
18763,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
30456,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
16476,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
30250,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [301]:
mca_ben = mca.mca(geo_sample)

ValueError: array must not contain infs or NaNs

In [265]:
pd.Series(mca_ben.L)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [120]:
mca_ben.expl_var(greenacre=False, N=4)

AttributeError: mca instance has no attribute 'expl_var'